# Least Squares Regression, SSR, RMSE, R-squared (Coefficient of Determination)

When you have a set of experimental data points $(x_i, y_i)$, where $i$ ranges from 1 to $n$ (the number of data points), and you want to find a mathematical function that best describes the relationship between $x$ and $y$, you are performing **curve fitting** or **regression analysis**. The goal is to find the parameters of a chosen function that make the function's output as close as possible to your observed $y$ values for the corresponding $x$ values.

Let's review some specific non-linear case, and consider that we want to approximate the data with the function:

$$f(x; A, B) = A \cdot (e^{-B \cdot x} - 1) + 100$$

Here, $A$ and $B$ are the parameters that we need to determine from the data points. The '100' is a constant offset in this specific function.

```{note}
As of now, we consider all experimental points with no errors. However, if $y$ values or both $x$ and $y$ values have some errors, we need to apply different algorithms. We will consider such algorihms later.
```

## The Core Idea: Minimizing Differences

> The fundamental idea behind most curve fitting methods is to minimize the "difference" between your experimental $y_i$ values and the $y$ values predicted by your chosen function, $f(x_i; A, B)$. This "difference" is often called the **residual**.

For each data point $(x_i, y_i)$, the residual, $e_i$, is defined as:

$e_i = y_i - f(x_i; A, B)$

Our goal is to find the values of $A$ and $B$ that make these residuals, collectively, as small as possible.

There are various methods for approximating data, but for continuous functions and without explicit error bars on individual points (as you specified initially), the most common and widely used method is **Least Squares Regression**.



## Least Squares Regression

The principle of least squares is to find the parameters (in our case, $A$ and $B$) that **minimize the sum of the squares of the residuals**. Why squares?
* Squaring the residuals ensures that positive and negative differences don't cancel each other out.
* It penalizes larger errors more heavily than smaller errors, which is often desirable.
* Mathematically, it leads to a convex optimization problem (for many functions), making it easier to find a unique minimum.

So, we want to minimize the following quantity, which is the **Sum of Squared Residuals (SSR)**:

$$ SSR(A, B) = \sum_{i=1}^{n} (y_i - f(x_i; A, B))^2 $$

Substituting our specific function:

$$ SSR(A, B) = \sum_{i=1}^{n} (y_i - (A \cdot (e^{-B \cdot x_i} - 1) + 100))^2 $$

To find the values of $A$ and $B$ that minimize $SSR$, we typically use calculus. We take the partial derivatives of $SSR$ with respect to each parameter ($A$ and $B$), set them equal to zero, and solve the resulting system of equations.

$$\frac{\partial SSR}{\partial A} = 0$$

$$\frac{\partial SSR}{\partial B} = 0$$

For linear regression, these equations are linear and have a direct analytical solution. However, for non-linear functions like ours (due to the $e^{-B \cdot x}$ term), these equations are often non-linear and require iterative numerical optimization algorithms (like the Levenberg-Marquardt algorithm, which is commonly used in `scipy.optimize.curve_fit` in Python). We won't derive the specific partial derivatives for the aforementioned function here, as it gets quite involved and typically handled by computational tools. The core idea remains the same: find $A$ and $B$ that make the slope of the $SSR$ surface zero.

Once you've found the best-fit parameters $A$ and $B$, you need to evaluate how "good" your approximation is. This is where metrics like SSR, RMSE, and R-squared come in.


## Sum of Squared Residuals (SSR)

As derived above, SSR is:

$$SSR = \sum_{i=1}^{n} (y_i - f(x_i; A, B))^2$$

**Meaning:** SSR is a direct measure of the total discrepancy between your observed data points and your fitted function. A smaller SSR indicates a better fit to the data.

**Understanding:**
* It's always non-negative.
* Its units are the square of the units of $y$.
* It's absolute: you CANNOT compare SSR directly between different datasets or models with different numbers of data points or vastly different scales of $y$.

## Root Mean Squared Error (RMSE)

RMSE is derived directly from SSR and is often more interpretable:

$$RMSE = \sqrt{\frac{SSR}{n}}$$

**Meaning:** RMSE represents the typical or average magnitude of the residuals. It gives you a sense of the average "error" your model makes in predicting $y$.

**Understanding:**
* It's in the same units as your dependent variable $y$. This makes it easier to interpret: "On average, our prediction is off by RMSE units of Y."
* It's sensitive to outliers because of the squaring of errors. Large errors contribute disproportionately to RMSE.
* Like SSR, a smaller RMSE indicates a better fit.
* You can compare RMSE values between different models **on the same dataset** to see which one performs better, provided the models have a similar number of parameters. Comparing RMSE across different datasets or datasets with vastly different scales of $y$ can still be misleading.

## R-squared (Coefficient of Determination)

R-squared is a very popular metric because it provides a standardized measure of how well your model explains the variability in the dependent variable $y$.

First, let's re-state the key sums of squares:

1.  **Total Sum of Squares (TSS):** This measures the total variability in the observed $y$ values around their mean $\bar{y}$. It represents how much the $y$ values vary in total, without considering any model.

    $$TSS = \sum_{i=1}^{n} (y_i - \bar{y})^2$$

    where $y_i$ are the observed data points and $\bar{y} = \frac{1}{n}\sum_{i=1}^{n} y_i$ is the mean of the observed $y$ values. TSS represents the total variability in the observed $y$ values around their mean. It's the sum of squared differences if you were to approximate all $y_i$ with their mean $\bar{y}$ (which is essentially a horizontal line).


2.  **Sum of Squared Residuals (SSR) / Sum of Squares Error (SSE) / Redisual Sum of Squares (RSS):** This measures the variability in the observed $y$ values that is *not* explained by the regression model. It's the sum of the squared differences between the observed $y_i$ and the predicted $f(x_i)$ (often denoted as $\hat{y}_i$).

    $$SSR = \sum_{i=1}^{n} (y_i - f(x_i))^2$$

    where $f(x_i)$ (or $\hat{y}_i$) are the predicted values from your model.

3.  **Sum of Squares due to Regression (SSR) / Explained Sum of Squares (ESS):** This measures the variability in the dependent variable ($y$) that *is* explained by the regression model. It's the sum of the squared differences between the predicted values $f(x_i)$ and the mean of the observed $y$ values $\bar{y}$.

    $$ESS = \sum_{i=1}^{n} (f(x_i) - \bar{y})^2$$

    *Note:* There can be some confusion with the acronym "SSR". In many statistical texts, "SSR" refers to "Sum of Squares due to Regression" (what we've called ESS here), and "SSE" refers to "Sum of Squared Errors" (what we've called SSR here). However, in the context of R-squared's formula ($1 - \frac{SSR}{TSS}$), the "SSR" in the numerator *always* refers to the **Sum of Squared Residuals/Errors**. To avoid confusion, we'll stick to $SSR_{\text{res}}$ for **Sum of Squared Residuals/Errors** and $ESS$ for the explained sum of squares.

R-squared is defined as:

$$R^2 = 1 - \frac{SSR_{\text{res}}}{TSS}$$

**Meaning:** R-squared tells you the proportion of the variance in the dependent variable ($y$) that is predictable from the independent variable ($x$) using your regression model. In simpler terms, it indicates how much of the variation in $y$ can be explained by your chosen function.

**Understanding:**
* $R^2$ ranges from 0 to 1 (or 0% to 100%).
* An $R^2$ of 1 (or 100%) means that your model perfectly explains all the variability in $y$. The residuals are all zero, and the function passes through every data point. This is rare in experimental data.
* An $R^2$ of 0 means that your model explains none of the variability in $y$. In this case, your model performs no better than simply predicting the mean of $y$ for all $x$ values.
* A higher $R^2$ generally indicates a better fit.

**Interpretation Caveats:**
* A high $R^2$ doesn't necessarily mean the model is "correct" or that the chosen function is the true underlying relationship. It just means it explains a lot of the variance.
* Adding more parameters to a model will generally increase $R^2$, even if those parameters don't significantly improve the model's predictive power (this is why **adjusted R-squared** is sometimes used, which penalizes for added complexity).
* **IMPORTANT**: $R^2$ is most appropriate for linear models. For non-linear models, using of $R^2$ is not recommended.
* It's possible to have a low $R^2$ for a valid model if the inherent variability in the data (noise) is very high, even if the model captures the underlying trend.

## Adjusted R-squared $R^2$

The standard R-squared $R^2$ measures the proportion of variance in the dependent variable that is explained by the independent variables in a regression model. While useful, $R^2$ has a significant drawback: **it always increases or stays the same when you add more independent variables (predictors) to your model, even if those new variables do not genuinely improve the model's explanatory power.** This can lead to misleading conclusions, as a more complex model might appear better simply because it has more terms, not because it's truly a better fit to the underlying phenomenon.

**Adjusted R-squared** addresses this issue by penalizing the inclusion of unnecessary predictors. It adjusts the $R^2$ value based on the number of predictors in the model and the number of data points.

Imagine you have a model with a certain $R^2$. If you add a new predictor that genuinely helps explain the variance in $y$, the $SSR_{\text{res}}$ will decrease significantly, and the $R^2$ will increase. However, if you add a new predictor that is irrelevant (e.g., random noise), $SSR_{\text{res}}$ will still decrease slightly (due to random chance or fitting noise), causing $R^2$ to increase, but this increase is not meaningful. Adjusted $R^2$ accounts for this by considering the degrees of freedom.

The formula for adjusted R-squared ($R^2_{adj}$) is:

$$R^2_{adj} = 1 - \frac{SSR_{\text{res}} / (n - k - 1)}{TSS / (n - 1)}$$

Let's break down the components and explain why this formula works:

* **$n$**: The number of data points (observations).
* **$k$**: The number of independent variables (predictors) in the model.
* **$SSR_{residuals}$**: Sum of Squared Residuals (unexplained variation).
* **$TSS$**: Total Sum of Squares (total variation).

To understand the division terms, we need to introduce the concept of **degrees of freedom (df)**:

* **Degrees of freedom for residuals ($df_{res}$):** This is the number of data points minus the number of parameters estimated by the model. For a model with $k$ independent variables and an intercept, there are $k+1$ parameters (the intercept and $k$ slopes/coefficients). So, $df_{res} = n - (k + 1)$.
* **Degrees of freedom for total variation ($df_{tot}$):** This is the number of data points minus 1 (because the mean $\bar{y}$ is estimated from the data). So, $df_{tot} = n - 1$.

Now, let's rewrite the formula using degrees of freedom:

$$R^2_{adj} = 1 - \frac{SSR_{\text{res}} / df_{res}}{TSS / df_{tot}}$$

This can also be expressed in terms of **Mean Squared Error (MSE)**:

* **Mean Squared Error of Residuals ($MSE_{res}$):** This is the average squared residual.
    $MSE_{res} = \frac{SSR_{\text{res}}}{n - k - 1}$
* **Mean Squared Total ($MST_{tot}$):** This is the sample variance of $y$.
    $MST_{tot} = \frac{TSS}{n - 1}$

Substituting these into the adjusted R-squared formula:

$$R^2_{adj} = 1 - \frac{MSE_{res}}{MST_{total}}$$


**Why this adjustment works:**

* **Penalizing Complexity:** When you add an irrelevant predictor, $SSR_{\text{res}}$ will decrease only slightly, but $k$ (the number of predictors) increases by 1. This means $n - k - 1$ (the denominator for $MSE_{res}$) decreases. If the decrease in $SSR_{\text{res}}$ is not substantial enough to offset the decrease in $n - k - 1$, then $MSE_{res}$ might actually *increase*. If $MSE_{res}$ increases, $R^2_{adj}$ will decrease. This is the "penalty" for adding useless variables.
* **Fair Comparison:** Adjusted $R^2$ allows for a more fair comparison between models with different numbers of predictors. A model with a higher adjusted $R^2$ is generally preferred, as it suggests a better fit that is not merely a result of adding more terms.

You can also derive adjusted R-squared from standard R-squared:

$$R^2 = 1 - \frac{SSR_{\text{res}}}{TSS}$$
$$\frac{SSR_{\text{res}}}{TSS} = 1 - R^2$$
$$SSR_{\text{res}} = (1 - R^2) \cdot TSS$$

Substitute $SSR_{\text{res}}$ into the adjusted R-squared formula:

$$R^2_{adj} = 1 - \frac{(1 - R^2) \cdot TSS / (n - k - 1)}{TSS / (n - 1)}$$

Cancel out $TSS$:

$$R^2_{adj} = 1 - (1 - R^2) \frac{(n - 1)}{(n - k - 1)}$$

This form clearly shows how $R^2_{adj}$ relates to $R^2$ and the degrees of freedom.

### Can Adjusted R-squared be Used for Linear vs. Non-Linear Functions?

**Yes, adjusted R-squared can be used for both linear and non-linear functions. However, it should be used with caution non-linear functions/models**

* **Linear Functions:** Adjusted R-squared is very commonly used in linear regression. It's the preferred metric over standard $R^2$ when comparing linear models with different numbers of predictors, as it helps identify models that are parsimonious (simple yet effective).

* **Non-Linear Functions:** The principles behind adjusted R-squared (penalizing for model complexity and providing a more robust measure of explained variance) apply equally to non-linear models, especially when they are fitted using the least squares method (for example, using `scipy.optimize.curve_fit`).

Technically, we can compute adjusted R-squared for any model fitted using least squares methods, including non-linear least squares.

However, for non-linear models, adjusted R-squared as well as R-squared can behave unexpectedly:
* It may not represent the proportion of variance explained in the same intuitive way
* It can sometimes be negative or exceed 1
* The penalty for additional parameters may not adequately capture model complexity in non-linear cases

Therefore, when you are comparing different non-linear models, or trying to decide if adding another parameter to your non-linear function is truly beneficial, adjusted R-squared is a more appropriate metric than the standard R-squared. However, the general recommendation is to use adjusted R-squared with caution for non-linear models.


## Summary of the Process (No Errors on X- and Y-axis Case):

1.  **Visualize your data:** Plot your $(x_i, y_i)$ points to get an initial sense of the trend. This helps in choosing an appropriate functional form.
2.  **Choose a functional form:** Based on your knowledge of the underlying process or by observing the data, select a mathematical function (like $A \cdot (e^{-B \cdot x} - 1) + 100$) that you believe can describe the relationship.
3.  **Define the objective function:** Formulate the Sum of Squared Residuals (SSR) equation, which is the quantity you want to minimize.
4.  **Minimize the SSR:** Use an optimization algorithm (like least squares implemented in `scipy.optimize.curve_fit`) to find the values of the parameters ($A$ and $B$ in the reviewed case) that minimize the SSR.
5.  **Evaluate the fit:** Calculate SSR, RMSE, and R-squared to quantify how well your chosen function with the optimized parameters fits your data.
6.  **Interpret the results:** Understand what the values of $A$, $B$, SSR, RMSE, and R-squared tell you about the relationship between $x$ and $y$ and the quality of your approximation.